In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import LineString
import os
from scipy.spatial import cKDTree
import networkx as nx
import osmnx as ox
import requests
import contextily as ctx
from numpy import int64
import ast

In [7]:
tiger_tracts10 = gpd.read_file('assignment/tiger/tiger_tracts_2010.geojson')
tiger_tracts10 = tiger_tracts10[['GEOID10', 'geometry']]
tiger_tracts10.drop_duplicates(subset=['GEOID10'], inplace=True)
tiger_tracts10.reset_index(inplace=True, drop=True)
tiger_tracts10.rename(columns={'GEOID10': 'GEOID'}, inplace=True)

tiger_tracts20 = gpd.read_file('assignment/tiger/tiger_tracts_2020.geojson')
tiger_tracts20 = tiger_tracts20[['GEOID', 'geometry']]

# population assignment

In [11]:
tiger_tracts20 = gpd.read_file('tiger_tracts_2020.geojson')
tiger_tracts20 = tiger_tracts20[['GEOID', 'geometry']]
tiger_tracts20['GEOID'] = tiger_tracts20['GEOID'].astype(int64)

In [7]:
from numpy import int64
from geopy.distance import geodesic
from scipy.spatial import cKDTree

def landfill_input_file(year):
    landfill = pd.read_csv('lanfill_FINAL.csv')
    before = landfill[landfill['Year Landfill Opened'] > year]
    landfill = landfill.drop(before.index)
    landfill = landfill.dropna(subset=['Latitude', 'Longitude'])
    landfill.drop_duplicates(subset=['Latitude', 'Longitude'], inplace=True)
    landfill.reset_index(inplace=True, drop=True)
    landfill_gdf = gpd.GeoDataFrame(landfill, geometry=gpd.points_from_xy(landfill.Longitude, landfill.Latitude), crs='EPSG:4269')
    landfill_gdf['facility_x'] = landfill_gdf.geometry.x
    landfill_gdf['facility_y'] = landfill_gdf.geometry.y
    landfill_gdf.rename(columns={'Landfill ID': 'facility_id'}, inplace=True)
    landfill_gdf_sub = landfill_gdf[['facility_id', 'facility_x', 'facility_y']]
    return landfill_gdf_sub

def landfill_input_file_heatmap(year):
    landfill = pd.read_csv('lanfill_FINAL.csv')
    before = landfill[landfill['Year Landfill Opened'] <= year]
    landfill = landfill.drop(before.index)
    landfill = landfill.dropna(subset=['Latitude', 'Longitude'])
    landfill.drop_duplicates(subset=['Latitude', 'Longitude'], inplace=True)
    landfill.reset_index(inplace=True, drop=True)
    landfill_gdf = gpd.GeoDataFrame(landfill, geometry=gpd.points_from_xy(landfill.Longitude, landfill.Latitude), crs='EPSG:4269')
    landfill_gdf['facility_x'] = landfill_gdf.geometry.x
    landfill_gdf['facility_y'] = landfill_gdf.geometry.y
    landfill_gdf.rename(columns={'Landfill ID': 'facility_id'}, inplace=True)
    landfill_gdf_sub = landfill_gdf[['facility_id', 'facility_x', 'facility_y']]
    return landfill_gdf_sub

def add_impacted_column(sample_facilities, sample_pfas, dist_threshold_miles):
    facilities_coords = list(zip(sample_facilities['facility_y'], sample_facilities['facility_x']))
    pfas_coords = list(zip(sample_pfas['ct_y'], sample_pfas['ct_x']))

    facility_tree = cKDTree(facilities_coords)
    nearby_facility_ids = []

    for pfas_point in pfas_coords:
        distances, indices = facility_tree.query(pfas_point, k=15)
        if distances[0] <= dist_threshold_miles:
            within_threshold = [sample_facilities.iloc[i]['facility_id'] for i in indices if geodesic(pfas_point, facilities_coords[i]).km <= dist_threshold_miles]
            nearby_facility_ids.append(within_threshold)
        else:
            nearby_facility_ids.append([])
    sample_pfas['nearby_facility_ids'] = nearby_facility_ids
    sample_pfas['impacted'] = sample_pfas['nearby_facility_ids'].apply(lambda x: len(x) > 0)
    return sample_pfas

def get_impacted_populations(year, dist_threshold):
    base_scenario = '01_CTAccuYear_High_StatusQuo'
    sample_pfas = pd.read_csv('Allocation to Census tracts_v3/Allocation to Census tracts_v3/{}_results/{}_{}_pfas_all_assigned.csv'.format(base_scenario[3:], base_scenario[3:], year), usecols=['GEOID', 'ct_x', 'ct_y'])
    sample_facilities = landfill_input_file(year)

    ct_pop = pd.read_csv("Census tracts' Population/Census tracts' Population/CT_Population_{}.csv".format(year))
    ct_pop['GEOID'] = ct_pop['GEO_ID'].str[9:]
    ct_pop['GEOID'] = ct_pop['GEOID'].astype(int64)
    sample_pfas = sample_pfas.merge(ct_pop, on='GEOID', how='left')
    sample_pfas = add_impacted_column(sample_facilities, sample_pfas, dist_threshold)
    
    # sample_pfas.to_csv('Allocation to Census tracts_v3/population_results/{}_{}km_pfas_all_assigned.csv'.format(year, dist_threshold), index=False)
    return print('Done processing {} {}.'.format(year, dist_threshold))

In [17]:
dist_thresholds = [1, 3, 5]
years = [2020, 2060]

for year in years:
    for dist_threshold in dist_thresholds:
        get_impacted_populations(year, dist_threshold)

Done processing 2020 1.
Done processing 2020 3.
Done processing 2020 5.
Done processing 2060 1.
Done processing 2060 3.
Done processing 2060 5.


In [8]:
def get_impacted_populations_annual(year, dist_threshold):
    base_scenario = '01_CTAccuYear_High_StatusQuo'
    sample_pfas = pd.read_csv('Allocation to Census tracts_v3/Allocation to Census tracts_v3/{}_results/{}_{}_pfas_all_assigned.csv'.format(base_scenario[3:], base_scenario[3:], year), usecols=['GEOID', 'ct_x', 'ct_y'])
    sample_facilities = landfill_input_file_heatmap(year)
    # sample_facilities = pd.read_csv('Allocation to Census tracts_v3/Allocation to Census tracts_v3/{}_results/{}_{}_MLF_pfas_assigned.csv'.format(base_scenario[3:], base_scenario[3:], year), usecols=['facility_id', 'facility_x', 'facility_y'])
    if year<2010:
        ct_pop = pd.read_csv("Census tracts' Population/Census tracts' Population/CT_Population_1990-2009.csv".format(year))
    elif year>2020:
        ct_pop = pd.read_csv("Census tracts' Population/Census tracts' Population/CT_Population_2022-2060.csv".format(year))
    else:
        ct_pop = pd.read_csv("Census tracts' Population/Census tracts' Population/CT_Population_{}.csv".format(year))
    ct_pop['GEOID'] = ct_pop['GEO_ID'].str[9:]
    ct_pop['GEOID'] = ct_pop['GEOID'].astype(int64)
    sample_pfas = sample_pfas.merge(ct_pop, on='GEOID', how='left')
    sample_pfas = add_impacted_column(sample_facilities, sample_pfas, dist_threshold)
    
    sample_pfas.to_csv('Allocation to Census tracts_v3/population_results/heatmaps/{}_{}km_pfas_all_assigned.csv'.format(year, dist_threshold), index=False)
    return print('Done processing {} {}.'.format(year, dist_threshold))

In [24]:
dists = [i for i in range(1, 11)]
for dist in dists:
    # get_impacted_populations_annual(1990, dist)
    # get_impacted_populations_annual(2000, dist)
    # get_impacted_populations_annual(2010, dist)
    get_impacted_populations_annual(2020, dist)
    get_impacted_populations_annual(2030, dist)
    get_impacted_populations_annual(2040, dist)
    get_impacted_populations_annual(2050, dist)
    get_impacted_populations_annual(2060, dist)

Done processing 2020 1.
Done processing 2030 1.
Done processing 2040 1.
Done processing 2050 1.
Done processing 2060 1.
Done processing 2020 2.
Done processing 2030 2.
Done processing 2040 2.
Done processing 2050 2.
Done processing 2060 2.
Done processing 2020 3.
Done processing 2030 3.
Done processing 2040 3.
Done processing 2050 3.
Done processing 2060 3.
Done processing 2020 4.
Done processing 2030 4.
Done processing 2040 4.
Done processing 2050 4.
Done processing 2060 4.
Done processing 2020 5.
Done processing 2030 5.
Done processing 2040 5.
Done processing 2050 5.
Done processing 2060 5.
Done processing 2020 6.
Done processing 2030 6.
Done processing 2040 6.
Done processing 2050 6.
Done processing 2060 6.
Done processing 2020 7.
Done processing 2030 7.
Done processing 2040 7.
Done processing 2050 7.
Done processing 2060 7.
Done processing 2020 8.
Done processing 2030 8.
Done processing 2040 8.
Done processing 2050 8.
Done processing 2060 8.
Done processing 2020 9.
Done processing 

In [16]:
landfill_accu = pd.read_csv('Allocation to Census tracts_v3\Re_ data and sample maps\landfill_Accum_High_StatusQuo_WO_Leachate.csv')
landfill_leachate = pd.read_csv('Allocation to Census tracts_v3\Re_ data and sample maps\LFLeachate_High_StatusQuo.csv')
ejscreen = pd.read_csv('EJSCREEN/EJSCREEN_2023_Tracts_with_AS_CNMI_GU_VI_csv/EJSCREEN_2023_Tracts_with_AS_CNMI_GU_VI.csv')

year = 2020
dists = [i for i in range(1, 11)]
for dist in dists:
    sample_f = pd.read_csv('Allocation to Census tracts_v3/population_results/heatmaps/{}_{}km_pfas_all_assigned.csv'.format(year, dist))
    sample_f['nearby_facility_ids'] = sample_f['nearby_facility_ids'].apply(ast.literal_eval)
    sample_f = sample_f[sample_f['impacted'] == True].explode('nearby_facility_ids')[['GEOID', 'ct_x', 'ct_y', 'nearby_facility_ids']]
    sample_f.reset_index(inplace=True, drop=True)
    sample_f['nearby_facility_ids'] = sample_f['nearby_facility_ids'].astype(str)
    sample_f.rename(columns={'nearby_facility_ids': 'facility_id'}, inplace=True)
    sample_f_accu = sample_f.merge(landfill_accu[['facility_id', '{}'.format(year)]], on='facility_id', how='left')
    sample_f_accu = sample_f_accu[['GEOID', '{}'.format(year)]].groupby('GEOID').sum().reset_index()
    sample_f_accu.rename(columns={'{}'.format(year): 'accu_pfas_within_{}'.format(dist)}, inplace=True)
    sample_f_leachate = sample_f.merge(landfill_leachate[['facility_id', '{}'.format(year)]], on='facility_id', how='left')
    sample_f_leachate = sample_f_leachate[['GEOID', '{}'.format(year)]].groupby('GEOID').sum().reset_index()
    sample_f_leachate.rename(columns={'{}'.format(year): 'leachate_pfas_within_{}'.format(dist)}, inplace=True)

ejscreen.dropna(subset=['GEOID'], inplace=True)
impacted_gids = ['accu_pfas_within_{}'.format(i) for i in range(1, 11)] + ['leachate_pfas_within_{}'.format(i) for i in range(1, 11)]
impacted_geoids = ejscreen[ejscreen[impacted_gids].sum(axis=1) > 0]['GEOID'].values
ejscreen = ejscreen[ejscreen['GEOID'].isin(impacted_geoids.tolist())]
ejscreen.reset_index(inplace=True, drop=True)
# ejscreen.to_csv('Allocation to Census tracts_v3/population_results/heatmaps/ejscreen_{}_impacted_cts.csv'.format(year), index=False)
# ejscreen.to_csv('Allocation to Census tracts_v3/population_results/heatmaps/ejscreen_{}_impacted_cts_km.csv'.format(year), index=False)

In [9]:
quantiles = ejscreen['accu_pfas_within_10'].quantile(np.arange(0, 1.1, 0.1))
ejscreen['quantile_range'] = pd.cut(ejscreen['accu_pfas_within_10'], bins=quantiles, labels=False, include_lowest=True)
ejscreen_og = pd.read_csv('EJSCREEN/EJSCREEN_2023_Tracts_with_AS_CNMI_GU_VI_csv/EJSCREEN_2023_Tracts_with_AS_CNMI_GU_VI.csv')

def hmap(fcol, fcol_label):
    heatmap_data = np.zeros((10, 10))  # 10x10 matrix for the heatmap
    for i in range(10):
        for j in range(10):
            # Filter rows in the j-th quantile
            quantile_rows = ejscreen[ejscreen['quantile_range'] == j]
            quantile_rows= quantile_rows[quantile_rows[f'accu_pfas_within_{i+1}'] > 0]
            # Calculate median income for the i-th pfas_within column within the j-th quantile
            median_income = quantile_rows[fcol].median()
            heatmap_data[j, i] = median_income

    center_val = ejscreen_og[fcol].median()
    ytlabels = ['[{:.1f}, {:.1f}]'.format(quantiles.tolist()[i], quantiles.tolist()[i+1]) for i in range(10)]
    ytlabels = ytlabels[::-1]
    sns.set(style='white', font_scale=1.5)
    plt.figure(figsize=(12, 10))
    heatmap_plot = sns.heatmap(heatmap_data[::-1], annot=True, fmt=".1f", cmap='RdYlGn_r', 
                # add the cells boundaries color to be white
                linecolor='white', linewidth=0.5,
                xticklabels=[f'{i}' for i in range(1, 11)], 
                # use quantiles list to label the y-axis
                yticklabels=ytlabels, 
                # vmin=0, 
                # vmax=100
                center=center_val
                )
    # plt.title('Median Income by PFAS Distance and Quantile')
    colorbar = heatmap_plot.collections[0].colorbar
    colorbar.set_label('Median {}'.format(fcol_label))
    plt.xlabel('Exposure Threshold (Miles)')
    plt.ylabel('Cumulated PFAS Range (kg)')

In [87]:
feature_cols = [
 'P_PEOPCOLORPCT',
 'P_LOWINCPCT',
 'P_UNEMPPCT',
 'P_RSEI_AIR',
 'P_D2_PM25',
#  'P_D2_OZONE',
 'P_D2_DSLPM',
#  'P_D2_CANCER',
 'P_D2_RESP',
#  'P_D2_RSEI_AIR',
 'P_D2_PTRAF',
 'P_D2_LDPNT',
 'P_D2_PNPL',
 'P_D2_PRMP',
#  'P_D2_PTSDF',
#  'P_D2_UST',
 'P_D2_PWDIS',
 ]
metric_vars = [
 'People of Color (%)',
 'Low Income (%)',
 'Unemployment (%)',
 'Toxic Releases to Air',
 'PM2.5',
#  'P_D2_OZONE',
 'Diesel PM',
#  'P_D2_CANCER',
 'Air Toxics Respiratory',
#  'P_D2_RSEI_AIR',
 'Traffic Proximity',
 'Lead Paint',
 'Superfund Proximity',
 'RMP Proximity',
#  'P_D2_PTSDF',
#  'P_D2_UST',
 'Wastewater Discharges',
 ]
# feature_cols = [f + '_y' for f in feature_cols]

In [10]:
base_scenario = '01_CTAccuYear_High_StatusQuo'

def ej_impacted_process(year):
    socio = gpd.read_file('tracts_w_pfas.geojson')
    dists = [i for i in range(1, 11)]
    impacted_gids = []
    for dist in dists:
        ipop_df = pd.read_csv('Allocation to Census tracts_v3/population_results/heatmaps/{}_{}km_pfas_all_assigned.csv'.format(year, dist))
        mlf_df = pd.read_csv('Allocation to Census tracts_v3/Allocation to Census tracts_v3/{}_results/{}_{}_MLF_pfas_assigned.csv'.format(base_scenario[3:], base_scenario[3:], year))
        wts_df = pd.read_csv('Allocation to Census tracts_v3/Allocation to Census tracts_v3/{}_results/{}_{}_WTS_pfas_assigned.csv'.format(base_scenario[3:], base_scenario[3:], year))

        ipop_df['nearby_facility_ids'] = ipop_df['nearby_facility_ids'].apply(ast.literal_eval)
        landfills = pd.concat([mlf_df[wts_df.columns], wts_df], axis=0)
        landfills.reset_index(inplace=True, drop=True)
        landfills['facility_id'] = landfills['facility_id'].astype(str)
        # landfills.drop_duplicates(inplace=True)

        ipop_df = ipop_df[ipop_df['impacted'] == True].explode('nearby_facility_ids')[['GEOID', 'ct_x', 'ct_y', 'nearby_facility_ids']]
        ipop_df.reset_index(inplace=True, drop=True)
        ipop_df['nearby_facility_ids'] = ipop_df['nearby_facility_ids'].astype(str)

        merged_df = ipop_df.merge(landfills, left_on='nearby_facility_ids', right_on='facility_id', how='left')
        merged_df.dropna(inplace=True)
        merged_df = merged_df[['GEOID', '{}'.format(year)]].groupby('GEOID')['{}'.format(year)].sum().reset_index()
        merged_df['GEOID'] = merged_df['GEOID'].astype(int64)
        merged_df.rename(columns={'{}'.format(year): 'pfas_within_{}'.format(dist)}, inplace=True)
        impacted_gids.append(merged_df['GEOID'].tolist())
        socio = socio.merge(merged_df, on='GEOID', how='left')

    ejscreen = pd.read_csv('D:/Users/hgazmeh/Codes/Datasets/EJSCREEN/EJSCREEN_2023_Tracts_with_AS_CNMI_GU_VI_csv/EJSCREEN_2023_Tracts_with_AS_CNMI_GU_VI.csv')
    ejscreen = ejscreen.merge(socio, left_on='ID', right_on='GEOID', how='right')
    for i in range(1, 11):
        ejscreen['pfas_within_{}'.format(i)] = ejscreen['pfas_within_{}'.format(i)].fillna(0)
    return ejscreen

In [14]:
for year in [1990, 2000, 2010, 2020, 2030, 2040, 2050, 2060]: # [1990, 2000, 2010, ]
    print('Processing {}...'.format(year))
    impacted_file = ej_impacted_process(year)
    # impacted_file.to_csv('Allocation to Census tracts_v3/population_results/heatmaps/ejscreen_impacted_{}_full.csv'.format(year), index=False)

Processing 1990...
Processing 2000...
Processing 2010...
Processing 2020...
Processing 2030...
Processing 2040...
Processing 2050...
Processing 2060...
